<a href="https://colab.research.google.com/github/PavlosPo/nlp-optimizers/blob/pavlos-playground/pytorch-experiments-1-fosi-adam/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FOSI Classes

In [1]:
# ! rm -rf ./fosi/
# !mkdir ./fosi/
# !unzip fosi.zip -d ./fosi/

Archive:  fosi.zip
 extracting: ./fosi/__init__.py      
   creating: ./fosi/jax_optim/
 extracting: ./fosi/jax_optim/__init__.py  
 extracting: ./fosi/jax_optim/extreme_spectrum_estimation.py  
 extracting: ./fosi/jax_optim/fosi_optimizer.py  
 extracting: ./fosi/jax_optim/lanczos_algorithm.py  
 extracting: ./fosi/jax_optim/lanczos_algorithm_sanity.py  
   creating: ./fosi/torch_optim/
 extracting: ./fosi/torch_optim/__init__.py  
 extracting: ./fosi/torch_optim/extreme_spectrum_estimation.py  
 extracting: ./fosi/torch_optim/fosi_optimizer.py  
 extracting: ./fosi/torch_optim/lanczos_algorithm.py  
 extracting: ./fosi/torch_optim/lanczos_algorithm_sanity.py  
 extracting: ./fosi/version.py       


## Working Example

In [ ]:
# !pip install torchopt
# !pip install datasets
# !pip install evaluate

In [2]:
import torch
import torchvision.transforms as transforms
from torch.utils.data.dataloader import default_collate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from torch.optim import Adam
import torch.nn as nn
import torchopt
import functorch
import evaluate
import torch.nn.functional as F

from datasets import load_dataset
from fosi import fosi_adam_torch

# # Set device
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# # Load pre-trained DistilBERT model and tokenizer
# base_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# base_model.to(device)
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [34]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding

# Load pre-trained BERT model and tokenizer
bert_model_name = "bert-base-uncased"
num_classes = 1
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=num_classes)
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

# # Freeze all parameters of the BERT model except for the last layer (classifier)
# for name, param in bert_model.named_parameters():
#     if 'classifier' not in name:  # Exclude parameters of the classifier layer
#         param.requires_grad = False

# Define a dense layer on top of the BERT model for classification
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask).logits
        probability = self.sigmoid(outputs.squeeze())
        return probability

# Instantiate the classifier
classifier = BertClassifier(bert_model, num_classes)

# Optionally, move the model to a GPU device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier.to(device)

# Now you can train the entire model (BERT + dense layer) on your task-specific data
# Make sure to prepare your data (input_ids, attention_mask, labels) using the BERT tokenizer
# and use an appropriate loss function and optimizer for your task


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertClassifier(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, 

In [29]:

# Define a function to preprocess the dataset
def prepare_dataset(example):
    return tokenizer(example['sentence'], add_special_tokens=True, truncation=True, padding=True, return_tensors='pt')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

dataset = load_dataset('glue', 'sst2').map(prepare_dataset, batched=True)
metric = evaluate.load("glue", "sst2")

# Split dataset into train and test sets, we use the train category because the test one has labels -1 only.
train_dataset = dataset['train'].select(range(0,500)).remove_columns(['sentence', 'idx']).rename_column('label', 'labels')
test_dataset = dataset['train'].select(range(500, 1000)).remove_columns(['sentence', 'idx']).rename_column('label', 'labels')


Map: 100%|██████████| 872/872 [00:00<00:00, 8629.73 examples/s]


In [7]:
len(dataset['train']['label'])

67349

In [30]:
# Define data loaders
batch_size = 128
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)

In [31]:
for i in (testloader):
  print(i['input_ids'])
  break

tensor([[ 101, 2980, 2006,  ...,    0,    0,    0],
        [ 101, 2149, 1037,  ...,    0,    0,    0],
        [ 101, 1037, 7815,  ...,    0,    0,    0],
        ...,
        [ 101, 2019, 7736,  ...,    0,    0,    0],
        [ 101, 8101, 2319,  ...,    0,    0,    0],
        [ 101, 1045, 2079,  ...,    0,    0,    0]])


In [32]:
# With buffers

def loss_fn(functional_model, params, buffers, input_ids, attention_mask, labels):
    preds = functional_model(params, buffers=buffers, input_ids=input_ids, attention_mask=attention_mask)
    loss = nn.functional.binary_cross_entropy(preds.squeeze().to(torch.float32), labels.squeeze().to(torch.float32))
    return loss

# def softmax_output_fn(params, buffers, input_ids, attention_mask, labels):
#     logits = model(params, buffers=buffers, input_ids=input_ids, attention_mask=attention_mask, labels=labels).logits
#     logits_tensor = torch.tensor(logits)
#     softmax_output = F.softmax(logits_tensor, dim=1)
#     return softmax_output

In [ ]:
# Without buffers

# def loss_fn(params, input_ids, attention_mask, labels):
#     loss = model(params, input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
#     return loss

# def softmax_output_fn(params, input_ids, attention_mask, labels):
#     logits = model(params, input_ids=input_ids, attention_mask=attention_mask, labels=labels).logits
#     logits_tensor = torch.tensor(logits)
#     softmax_output = F.softmax(logits_tensor, dim=1)
#     return softmax_output

In [40]:
# Train the model
classifier.train()
for epoch in range(2):
    for i, data in enumerate(trainloader, 0):
        print("\n")
        print("*"*100)
        print("\n")

        if i == 0 and epoch == 0: # Initialize optimizer and model parameters
            print(f"input_ids: {data['input_ids']}")
            print(f"attention_mask: {data['attention_mask']}")
            print(f"labels: {data['labels']}")
            # Define optimizer
            base_optimizer = torchopt.adam(lr=0.01)
            optimizer = fosi_adam_torch(base_optimizer, loss_fn, data, num_iters_to_approx_eigs=500, alpha=0.01)
            # func_optimizer = torchopt.FuncOptimizer(fosi_adam_torch(base_optimizer, loss_fn, data, num_iters_to_approx_eigs=500, alpha=0.01))
            model, params, buffers = functorch.make_functional_with_buffers(model=classifier)
            opt_state = optimizer.init(params)

        input_ids = data['input_ids'].squeeze().to(device)
        attention_mask = data['attention_mask'].squeeze().to(device)
        labels = data['labels'].squeeze().to(device)
        print(f"Labels: \n{labels}\n")

        loss = loss_fn(functional_model=model, 
                       params=params, buffers=buffers, input_ids=input_ids,attention_mask=attention_mask, labels=labels)
        print(f"Step: {i}\n")
        print(f"Loss: {loss}\n")

        # func_optimizer.step(loss, params, inplace=True)

        # Calculate gradients
        grads = torch.autograd.grad(loss, params)

        # Update model parameters
        updates, opt_state = optimizer.update(grads, opt_state, params)
        params = torchopt.apply_updates(params, updates, inplace=True)

        # Should this method exists? Are buffers neseccary to update?
        # Update buffers based on updated parameters
        # buffers = torchopt.update_buffers(model, params, buffers)
    evaluation_results = []

    # Evaluate the model
    classifier.eval()
    with torch.no_grad():
        for i, data in enumerate(testloader):
            evaluation_result = {}

            input_ids = data['input_ids'].squeeze().to(device)
            attention_mask = data['attention_mask'].squeeze().to(device)
            labels = data['labels'].squeeze().to(device)

            preds = model(params, buffers, input_ids, attention_mask=attention_mask)

            predicted_labels = torch.round(preds)

            # Save the evaluation results
            evaluation_result['input_ids'] = input_ids.cpu().tolist()
            evaluation_result['attention_mask'] = attention_mask.cpu().tolist()
            evaluation_result['labels'] = labels.cpu().tolist()
            evaluation_result['preds'] = preds.cpu().tolist()
            evaluation_result['predicted_label'] = predicted_labels.cpu().tolist()

            evaluation_results.append(evaluation_result)
            metric.add_batch(predictions=predicted_labels, references=labels)

print(f'Epoch: {epoch}')
results = metric.compute()
print(f"Results: \n{results}\n")

print('Finished Training')




****************************************************************************************************


input_ids: tensor([[  101,  1996,  1010,  ...,     0,     0,     0],
        [  101,  8579,  4244,  ...,     0,     0,     0],
        [  101,  3288, 14095,  ...,     0,     0,     0],
        ...,
        [  101,  2387,  2129,  ...,     0,     0,     0],
        [  101,  2079,  5051,  ...,     0,     0,     0],
        [  101,  1996,  2466,  ...,     0,     0,     0]])
attention_mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
labels: tensor([0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
        1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
        1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,

In [41]:
print(evaluation_results[0]['labels'])

[0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0]


In [42]:
print(evaluation_results[0].keys())

dict_keys(['input_ids', 'attention_mask', 'labels', 'preds', 'predicted_label'])


In [43]:
print(evaluation_results[0]['preds'])

[0.7217391133308411, 0.6180686354637146, 0.6812028884887695, 0.6741484999656677, 0.7031254768371582, 0.7577357888221741, 0.6773779988288879, 0.616105854511261, 0.6453668475151062, 0.6326610445976257, 0.7034841775894165, 0.7196648120880127, 0.6296562552452087, 0.7020332217216492, 0.6612133979797363, 0.5831028819084167, 0.6385987997055054, 0.7122625708580017, 0.6820878386497498, 0.7093048691749573, 0.718841552734375, 0.6197752356529236, 0.6700179576873779, 0.6960123181343079, 0.6805780529975891, 0.6975422501564026, 0.6149519085884094, 0.5524992346763611, 0.7058302760124207, 0.6343899369239807, 0.6984028816223145, 0.6195926666259766, 0.6115147471427917, 0.6497581601142883, 0.5942254662513733, 0.5956348776817322, 0.5560219287872314, 0.58880615234375, 0.7032749056816101, 0.5823532342910767, 0.6333704590797424, 0.6251976490020752, 0.6919683218002319, 0.7145364284515381, 0.7224327921867371, 0.5919395089149475, 0.6705578565597534, 0.7285133004188538, 0.6557971239089966, 0.7236104607582092, 0.6

In [44]:
print(evaluation_results[0]['predicted_label'])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [45]:
import gc
gc.collect() # Python thing
torch.cuda.empty_cache() # PyTorch thing

In [ ]:
# # Train the model
# for epoch in range(3):
#     for i, data in enumerate(trainloader, 0):
#         print("\n")
#         print("*"*100)
#         print("\n")

#         if i == 0: # Initialize optimizer and model parameters
#             print(f"input_ids: {data['input_ids']}")
#             print(f"attention_mask: {data['attention_mask']}")
#             print(f"labels: {data['labels']}")
#             # Define optimizer
#             base_optimizer = torchopt.adam(lr=0.01)
#             optimizer = fosi_adam_torch(base_optimizer, loss_fn, data, num_iters_to_approx_eigs=500, alpha=0.01)
#             model, params, buffers = functorch.make_functional_with_buffers(model=base_model)
#             # model, params = functorch.make_functional(model=base_model)
#             opt_state = optimizer.init(params)
#             # model.train()


#         input_ids = data['input_ids']
#         attention_mask = data['attention_mask']
#         labels = data['labels']
#         print(f"Labels: \n{labels}\n")

#         loss = loss_fn(params, buffers, input_ids, attention_mask, labels)
#         # loss = loss_fn(params, input_ids, attention_mask, labels)
#         print(f"Step: {i}\n")
#         print(f"Loss: {loss}\n")

#         print(f"Calculating Gradients\n")
#         grads = torch.autograd.grad(loss, params)
#         # print(f"Grads: \n{grads}\n")
#         print(f"Calculating updates in the model...\n")
#         updates, opt_state = optimizer.update(grads, opt_state, params)
#         print("Applying updates\n")
#         params = torchopt.apply_updates(params, updates, inplace=True)

#     # Evaluate the model
#     # model.eval()
#     with torch.no_grad():
#         for data in testloader:
#             input_ids = data['input_ids'].to(device)
#             attention_mask = data['attention_mask'].to(device)
#             labels = data['labels'].to(device)

#             logits = model(params, buffers, input_ids, attention_mask=attention_mask).logits
#             # logits = model(params, input_ids, attention_mask=attention_mask).logits
#             print(f"Logits: {logits}")
#             logits_tensor = torch.tensor(logits)
#             softmax_output = F.softmax(logits_tensor, dim=1)

#             print(f"SoftMax of Logits: {softmax_output}")
#             predicted_labels = torch.argmax(softmax_output, dim=1)
#             print(f"Argmax Predictions: {predicted_labels}")

#             metric.add_batch(predictions=predicted_labels, references=labels)


#     print(f'Epoch: {epoch}')
#     results = metric.compute()
#     print(f"Results: \n{results}\n")
#     # print(f'Correct: {correct}\n')
#     # print(f'Total: {total}\n')
#     # print(f'Predicted: {predicted}\n')
#     # print(f'outputs: {outputs}\n')
#     # print(f'labels: {labels}\n')
#     # print(f'Accuracy of the network on the test data: {100 * correct / total}%')
#     # model.train() # for the next epoch

# print('Finished Training')


In [ ]:
# # Evaluate the model
# model.eval()
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         input_ids = data['input_ids'].to(device)
#         attention_mask = data['attention_mask'].to(device)
#         labels = data['labels'].to(device)
#         outputs = model(params, buffers, input_ids, attention_mask=attention_mask)
#         _, predicted = torch.max(outputs.logits, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

In [ ]:
# # Define training loop
# for epoch in range(3):  # Adjust number of epochs as needed
#     model.train()
#     for i, batch in enumerate(train_dataset):
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['label']

#         #This is taking care automatically
#         # optimizer.zero_grad() This is taking care automatically

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#         # if (i + 1) % 100 == 0:
#         #     print(f'[{epoch + 1}, {i + 1:5d}] loss: {loss.item():.3f}')

#     # Evaluate on test set
#     model.eval()
#     total = 0
#     correct = 0
#     with torch.no_grad():
#         for batch in test_dataset:
#             input_ids = batch['input_ids']
#             attention_mask = batch['attention_mask']
#             labels = batch['label']
#             print("Labels: \n", labels)

#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             logits = outputs.logits
#             _, predicted = torch.max(logits, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     accuracy = correct / total
#     print(f'Test accuracy: {accuracy:.4f}')

# print('Finished Training')
